### 用于特征提取和图像匹配变换

In [12]:
import cv2
import numpy as np
import PIL
from PIL import Image

In [18]:
# 1. 特征提取
def extract_features(image):
    # image_array = np.array(image)
    feature_extractor = cv2.SIFT_create()
    keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
    return keypoints, descriptors

# 2. 特征匹配
def match_features_bf(descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

def match_features_flann(descriptors1, descriptors2):
    # FLANN 参数设置
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    # 创建 FLANN 匹配器
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    # 使用 KNN 进行匹配
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    # 选择好的匹配
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)

    return good_matches
# 注掉的方法是基于透视变换
# # 3. 变换估计
def estimate_transform(keypoints1, keypoints2, good_matches):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    # 使用RANSAC算法估计变换矩阵
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return M

#4. 变换应用
def apply_transform(image, M):
    # 应用估计的变换矩阵到图像
    transformed_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))
    return transformed_image

# 3. 仿射变换估计
# def estimate_transform(keypoints1, keypoints2, good_matches):
#     src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#     dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#     # 使用RANSAC算法估计仿射变换矩阵
#     M = cv2.estimateAffinePartial2D(src_pts, dst_pts)[0]
#     return M

# # 4. 仿射变换应用
# def apply_transform(image, M):
#     transformed_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
#     return transformed_image

In [ ]:
# 一次配准
image1 = cv2.imread(r'images\2.png', cv2.IMREAD_GRAYSCALE).astype(np.uint8)
image2 = cv2.imread(r'images\222.png', cv2.IMREAD_GRAYSCALE).astype(np.uint8)
# array = np.load('anti_mean.npy')
# image1 = np.clip(image1 / array,0,255)
# image2 = np.clip(image2 / array,0,255)
# image1 = image1.astype(np.uint8)
# image2 = image2.astype(np.uint8)
assert image1.shape == image2.shape

In [ ]:
# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)
# 2. 特征匹配

good_matches = match_features_bf(descriptors1, descriptors2)
print(len(good_matches))

In [ ]:
# 3. 变换估计
M = estimate_transform(keypoints1, keypoints2, good_matches)
# 4. 变换应用
transformed_image = apply_transform(image1, M)
cv2.imwrite('trans_image/transformed2to222.png', transformed_image)

### 后面是SIFT和BFMatch合理性判断

In [22]:
#一次配准
image1 = cv2.imread(r'image\1.png', cv2.IMREAD_GRAYSCALE).astype(np.uint8)
image2 = cv2.imread(r'image\2.png', cv2.IMREAD_GRAYSCALE).astype(np.uint8)

In [23]:
# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1,descriptors2, k=2)
good_matches = match_features_bf(descriptors1, descriptors2)
img5 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,matches,None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
pillow_image = Image.fromarray(cv2.cvtColor(img5, cv2.COLOR_BGR2RGB))
pillow_image.show()

In [24]:
print(matches)
print(good_matches)
print([good_matches])

((< cv2.DMatch 000002205BF5C9D0>, < cv2.DMatch 000002205FC76D70>), (< cv2.DMatch 000002205FC75D70>, < cv2.DMatch 000002205FC77D90>), (< cv2.DMatch 000002205FC75B70>, < cv2.DMatch 000002205FC763D0>), (< cv2.DMatch 000002205FC76DB0>, < cv2.DMatch 000002205FC75610>), (< cv2.DMatch 000002205FC75D90>, < cv2.DMatch 000002205FC76350>), (< cv2.DMatch 000002205FC76DF0>, < cv2.DMatch 000002205FC75870>), (< cv2.DMatch 000002205FC76E30>, < cv2.DMatch 000002205FC76E90>), (< cv2.DMatch 000002205FC76670>, < cv2.DMatch 000002205FC766F0>), (< cv2.DMatch 000002205FC759D0>, < cv2.DMatch 000002205FC76610>), (< cv2.DMatch 000002205FC762F0>, < cv2.DMatch 000002205FC757F0>), (< cv2.DMatch 000002205FC75910>, < cv2.DMatch 000002205FC75BB0>), (< cv2.DMatch 000002205FC75D30>, < cv2.DMatch 000002205FC75810>), (< cv2.DMatch 000002205FC75710>, < cv2.DMatch 000002205FC75DB0>), (< cv2.DMatch 000002205FC75F70>, < cv2.DMatch 000002205FC76070>), (< cv2.DMatch 000002205FC76030>, < cv2.DMatch 000002205FC760B0>), (< cv2.DM

In [33]:
img6 = cv2.drawMatchesKnn(image1,keypoints1,image2,keypoints2,[good_matches],None,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS | cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
pillow_image = Image.fromarray(cv2.cvtColor(img6, cv2.COLOR_BGR2RGB))
pillow_image.show()

In [ ]:
pillow_image.save('image/1_2m.png')

In [31]:
def crop_black_borders(image):
    if len(image.shape) == 3 and image.shape[2] == 3:  # 彩色图像
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif len(image.shape) == 2:  # 灰度图像
        gray = image
    else:
        raise ValueError("Unsupported image format")
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        return image[y:y+h, x:x+w]
    return image

M = estimate_transform(keypoints1, keypoints2, good_matches)
result = cv2.warpPerspective(image1, M, (image1.shape[1] + image2.shape[1], image1.shape[0]+image2.shape[0]))
result[0:image2.shape[0], 0:image2.shape[1]] = image2
result = crop_black_borders(result)
result= PIL.Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
result.show()